# Keras: Time Series Anomaly
***

This Jupyter notebook will contain a clear and concise overview of how to load CAO points information from the CAO website into a pandas data frame, a detailed comparison of the level 8 CAO points in 2019, 2020, and 2021 and some visualisations to enhance the viewer experience.

## Introduction


## Conclusion

# End
***